In [1]:
#%pip install peft
#%pip install --upgrade datasets tokenizers

In [1]:
import random
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from peft import (
    LoraConfig,
    PromptTuningConfig,
    PromptTuningInit,
    get_peft_model,
    TaskType,
)

2025-05-15 21:29:29.016568: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-15 21:29:29.030364: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747344569.048330    8918 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747344569.053930    8918 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-15 21:29:29.071378: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

Using the latest cached version of the module from /home/sagemaker-user/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Wed May 14 23:45:49 2025) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from /home/sagemaker-user/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--f1/34c46321f42186df33a6260966e34a368f14868d9cc2ba47d142112e2800d233 (last modified on Thu May 15 15:15:13 2025) since it couldn't be found locally at evaluate-metric--f1, or remotely on the Hugging Face Hub.


### Data loading & formatting

In [3]:
def load_and_prepare_binary(path: str, test_size: float = 0.1, seed: int = 42):
    random.seed(seed)
    df = pd.read_csv(path, sep=';', usecols=['Premise', 'QCC', 'CorrectAnswer', 'Answer1', 'Answer2'])
    examples = []
    for _, row in df.iterrows():
        text_base = f"Premise: {row['Premise']} Question: {row['QCC']}"
        choices = [row['CorrectAnswer'], row['Answer1'], row['Answer2']]
        for choice in choices:
            label = 1 if choice == row['CorrectAnswer'] else 0
            examples.append({
                'text': f"{text_base} Choice: {choice}",
                'label': label
            })
    ds = Dataset.from_pandas(pd.DataFrame(examples))
    return ds.train_test_split(test_size=test_size)

### Tokenizer & tokenization util

In [4]:
def get_tokenizer_and_collator(model_name: str = 'bert-base-uncased'):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    collator = DataCollatorWithPadding(tokenizer)
    return tokenizer, collator

def tokenize_dataset(tokenizer):
    def preprocess(batch):
        tokenized = tokenizer(batch['text'], truncation=True, padding=False)
        tokenized['label'] = batch['label']
        return tokenized
    return preprocess

### Metrics

In [5]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1_score, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1_score}

### Trainer builders

In [6]:
def get_lora_sequence_trainer(model_name, tokenizer, collator, train_ds, eval_ds,
                               output_dir, epochs=20, train_batch=16, eval_batch=32,
                               lora_r=4, lora_alpha=16, lora_dropout=0.1):
    base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    peft_config = LoraConfig(
        task_type= TaskType.SEQ_CLS,
        target_modules=['query', 'key'],
        inference_mode=False,
        r=lora_r,
        lora_alpha=lora_alpha,
        lora_dropout=lora_dropout
    )
    model = get_peft_model(base_model, peft_config)
    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=train_batch,
        per_device_eval_batch_size=eval_batch,
        eval_strategy='epoch',
        save_strategy='epoch',
        save_total_limit=2,
        logging_steps=5,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy'
    )
    return Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics
    )

In [7]:
def get_prompt_sequence_trainer(model_name, tokenizer, collator, train_ds, eval_ds,
                                 output_dir, epochs=100, train_batch=16, eval_batch=32,
                                 num_virtual_tokens=5):
    base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    peft_config = PromptTuningConfig(
        task_type= TaskType.SEQ_CLS,
        prompt_tuning_init=PromptTuningInit.RANDOM,
        num_virtual_tokens=num_virtual_tokens,
        tokenizer_name_or_path=tokenizer.name_or_path
    )
    model = get_peft_model(base_model, peft_config)
    print(model.print_trainable_parameters())
    
    args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=train_batch,
        per_device_eval_batch_size=eval_batch,
        eval_strategy='epoch',
        save_strategy='epoch',
        save_total_limit=2,
        logging_steps=5,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy'
    )
    return Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics
    )

### Inference

In [8]:
def predict_sequence(trainer, texts, max_length=512):
    model = trainer.model
    model.eval()

    # Correct tokenizer
    tokenizer = trainer.processing_class

    # Tokenize
    enc = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )

    # Device‐align
    device = next(model.parameters()).device
    enc = {k: v.to(device) for k, v in enc.items()}

    # Forward pass
    with torch.no_grad():
        logits = model(**enc).logits
    
    import torch.nn.functional as F
    probability = F.sigmoid(logits)
    
    # Predictions
    return probability
    #torch.argmax(logits, dim=1).tolist()

### Running

In [9]:
DATA_PATH = './data/raw/CRASS_FTM_main_data_set.csv'
OUTPUT_PATH_PROMPT = './causal-classifier-prompt'
OUTPUT_PATH_LORA = './causal-classifier-lora'
BASE_MODEL = "bert-base-uncased"

##### Load and Tokenize

In [10]:
ds_splits = load_and_prepare_binary(DATA_PATH)
train_ds, test_ds = ds_splits['train'], ds_splits['test']

In [11]:
tokenizer, collator = get_tokenizer_and_collator(BASE_MODEL)

In [12]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
examples = ds_splits["train"][:6]
features = tokenize_dataset(tokenizer)(examples)

In [14]:
idx = 2
[tokenizer.decode(features["input_ids"][idx+ix]) for ix in range(3)]

['[CLS] premise : a car speeds down a road. question : what would have happened if the car had slowed down on the road? choice : nothing would have happened. [SEP]',
 '[CLS] premise : a dog is in a house. question : what would have happened if the dog had not been in the house? choice : the dog would have been inside the house. [SEP]',
 '[CLS] premise : a man buys a hat. question : what would have happened if the man had sold the hat? choice : he would have lost money. [SEP]']

In [15]:
train_ds = train_ds.map(tokenize_dataset(tokenizer), batched=True)
test_ds = test_ds.map(tokenize_dataset(tokenizer), batched=True)
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/739 [00:00<?, ? examples/s]

Map:   0%|          | 0/83 [00:00<?, ? examples/s]

#####  Baseline evaluation (no fine-tuning)

In [16]:
print("Baseline evaluation (no training)")
base_model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, num_labels=2)
base_trainer = Trainer(
    model=base_model,
    args=TrainingArguments(
        per_device_eval_batch_size=32,
        do_train=False
    ),
    eval_dataset=test_ds,
    processing_class=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics
)
print(base_trainer.evaluate())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Baseline evaluation (no training)


{'eval_loss': 0.6301443576812744, 'eval_model_preparation_time': 0.0026, 'eval_accuracy': 0.6867469879518072, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_runtime': 0.454, 'eval_samples_per_second': 182.805, 'eval_steps_per_second': 6.607}


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2025/05/15 21:30:02 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id 06065e7405e041ee9745b882839f2b90: Failed to log run data: Exception: Changing param values is not allowed. Param with key='logging_dir' was already logged with value='trainer_output/runs/May15_21-29-56_default' for run ID='06065e7405e041ee9745b882839f2b90'. Attempted logging new value './causal-classifier-prompt/runs/May15_21-30-02_default'.
2025/05/15 21:30:03 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id 06065e7405e041ee9745b882839f2b90: Failed to log run data: Exception: Changing param values is not allowed. Param with key='problem_type' was already logged with value='single_label_clas

##### Prompt Tuning

In [17]:
print("Prompt tuning for sequence classification")
prompt_trainer = get_prompt_sequence_trainer(
    BASE_MODEL, tokenizer, collator, train_ds, test_ds, OUTPUT_PATH_PROMPT
)
prompt_trainer.train()
print(prompt_trainer.evaluate())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prompt tuning for sequence classification
trainable params: 3,840 || all params: 109,487,618 || trainable%: 0.0035
None


/tmp/ipykernel_8918/290341359.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.681500,0.679449,0.662651,0.437500,0.269231,0.333333
2,0.708400,0.678583,0.650602,0.384615,0.192308,0.256410
3,0.704100,0.677809,0.638554,0.333333,0.153846,0.210526
4,0.692000,0.677059,0.650602,0.363636,0.153846,0.216216
5,0.705600,0.676218,0.662651,0.400000,0.153846,0.222222
6,0.696700,0.675479,0.662651,0.400000,0.153846,0.222222
7,0.678300,0.674686,0.650602,0.333333,0.115385,0.171429
8,0.678800,0.673828,0.662651,0.375000,0.115385,0.176471
9,0.674700,0.673033,0.650602,0.285714,0.076923,0.121212
10,0.666300,0.672275,0.650602,0.285714,0.076923,0.121212


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

{'eval_loss': 0.6616513133049011, 'eval_accuracy': 0.6867469879518072, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_runtime': 0.2148, 'eval_samples_per_second': 386.365, 'eval_steps_per_second': 13.965, 'epoch': 100.0}


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [43]:
prompt_trainer.save_model(OUTPUT_PATH_PROMPT)

In [20]:
print("LoRA tuning for sequence classification")
lora_trainer = get_lora_sequence_trainer(
    BASE_MODEL, tokenizer, collator, train_ds, test_ds, OUTPUT_PATH_LORA
)
lora_trainer.train()
print(lora_trainer.evaluate())

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LoRA tuning for sequence classification


/tmp/ipykernel_7500/2840488858.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  return Trainer(
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.719300,0.601047,0.698795,0.000000,0.000000,0.000000
2,0.626700,0.593062,0.698795,0.000000,0.000000,0.000000
3,0.654900,0.576818,0.698795,0.000000,0.000000,0.000000
4,0.543900,0.556212,0.698795,0.000000,0.000000,0.000000
5,0.603200,0.543652,0.686747,0.333333,0.040000,0.071429
6,0.541200,0.539728,0.650602,0.333333,0.160000,0.216216
7,0.560300,0.535577,0.650602,0.333333,0.160000,0.216216
8,0.572800,0.534732,0.662651,0.411765,0.280000,0.333333
9,0.582700,0.536209,0.650602,0.400000,0.320000,0.355556
10,0.529400,0.534600,0.650602,0.400000,0.320000,0.355556


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

{'eval_loss': 0.6010472178459167, 'eval_accuracy': 0.6987951807228916, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_runtime': 0.1812, 'eval_samples_per_second': 457.978, 'eval_steps_per_second': 16.553, 'epoch': 20.0}


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
lora_trainer.save_model(OUTPUT_PATH_LORA)

In [22]:
texts = ["premise: A girl kisses a boy. question: What would have happened if the girl had killed the boy? choice: She would have been liable to prosecution.",
"premise:  A girl kisses a boy. question: What would have happened if the girl had killed the boy? The boy would have been arrested for assault",
"premise: A girl kisses a boy. question: What would have happened if the girl had killed the boy? choice: The boy would have kissed the girl"]

predict_sequence(lora_trainer,texts)

tensor([[0.5016, 0.3526],
        [0.5235, 0.3329],
        [0.5301, 0.3208]], device='cuda:0')

In [23]:
#for name, module in base_model.named_modules():
#    print(name)
#    print(module)

In [32]:
from peft import PeftModel
from transformers import AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

def predict_prompt_sequence(
    prompt_tuned_dir: str,
    base_model_name: str,
    tokenizer,
    texts: list[str],
    max_length: int = 492,# 512 earlier max length minus 20 virtual tokens
    num_virtual_tokens: int = 20,    # MUST match what you used during training
    device: torch.device | None = None,
):
    """
    Load a prompt-tuned model and run inference.  We first bump up
    the position embeddings by num_virtual_tokens so that
    512 + num_virtual_tokens positions are supported.
    """
    # 1) Load the frozen base model
    base_model = AutoModelForSequenceClassification.from_pretrained(
        base_model_name,
        num_labels=2,
    )
    
    # 3) Inject the prompt-tuned weights
    model = PeftModel.from_pretrained(base_model, prompt_tuned_dir)
    model.eval()

    # 4) Move to device
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # 5) Tokenize normally (no need to add “[VIRT_0]...[VIRT_19]” yourself)
    enc = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=max_length,
        return_tensors="pt",
    )
    enc = {k: v.to(device) for k, v in enc.items()}

    # 6) Forward pass
    with torch.no_grad():
        logits = model(**enc).logits

    # 7) Convert to probabilities
    probs = torch.softmax(logits, dim=-1)
    
    # ——— 1) Find the ModuleDict that contains the prompt encoder ———
    prompt_enc_dict = None
    for name, module in model.named_modules():
        if name.endswith("prompt_encoder"):
            prompt_enc_dict = module
            break
    assert prompt_enc_dict is not None, "Prompt-encoder not found!"
    
    # ——— 2) Extract the actual PromptEmbedding inside the dict ———
    # It usually has a single key ("default"), but this is robust:
    prompt_encoder = None
    for sub in prompt_enc_dict.children():
        # first child should be PromptEmbedding
        prompt_encoder = sub
        break
    assert prompt_encoder is not None, "No PromptEmbedding inside ModuleDict!"
    
    # ——— 3) Pull out the soft-prompt embeddings ———
    # PromptEmbedding has an `.embedding` module
    soft_embs: torch.Tensor = prompt_encoder.embedding.weight
    # shape = (num_virtual_tokens, hidden_size)
    
    # ——— 4) Get the base model’s real token embeddings ———
    base_embs: torch.Tensor = model.base_model.get_input_embeddings().weight
    # shape = (vocab_size, hidden_size)
    
    # ——— 5) Compute cosine similarities & top-k neighbors ———
    V, H = soft_embs.shape
    T, _ = base_embs.shape
    soft_norm = F.normalize(soft_embs, dim=1)  # (V, H)
    base_norm = F.normalize(base_embs, dim=1)  # (T, H)
    
    sims = torch.matmul(soft_norm, base_norm.t())  # (V, T)
    top_k = 5
    values, indices = sims.topk(top_k, dim=1)      # (V, K)
    
    # ——— 6) Decode & print ———
    for i in range(V):
        toks = tokenizer.convert_ids_to_tokens(indices[i].tolist())
        scores = [f"{v:.3f}" for v in values[i].tolist()]
        print(f"Virtual token #{i:2d} →", ", ".join(f"{tok}({sc})" for tok, sc in zip(toks, scores)))
            
    return probs.cpu()

In [40]:
prompt_probs = predict_prompt_sequence(
    prompt_tuned_dir=OUTPUT_PATH_PROMPT,
    base_model_name=BASE_MODEL,
    tokenizer=tokenizer,
    texts=texts,
)
print("Prompt-Tuned Probabilities:\n", prompt_probs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Virtual token # 0 → ##urbed(0.099), british(0.098), marquez(0.095), united(0.092), fang(0.092)
Virtual token # 1 → republic(0.123), ##ality(0.115), ##ur(0.114), duo(0.113), florida(0.107)
Virtual token # 2 → adam(0.092), cancel(0.088), factors(0.088), walter(0.086), ##gg(0.084)
Virtual token # 3 → premier(0.143), morgan(0.132), cup(0.129), trophy(0.129), tamil(0.123)
Virtual token # 4 → trees(0.123), selections(0.119), ##oted(0.114), sonya(0.113), buttons(0.111)
Prompt-Tuned Probabilities:
 tensor([[0.5348, 0.4652],
        [0.4974, 0.5026],
        [0.5341, 0.4659]])
